In [ ]:
import os, sys
project_root_dir = os.path.join(os.getcwd(),'..')
if project_root_dir not in sys.path:
    sys.path.append(project_root_dir)

import torch, config
import IPDL
from autoencoder import SDAE, SDAE_TYPE, AE_CONV_UPSAMPLING
import numpy as np

# FMNIST

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor, Resize, Compose, Normalize

transforms = Compose([
    Resize((64,64)),
    ToTensor(),
    Normalize([0.5], [0.5])
])

# train_set = datasets.FashionMNIST("dataset", train=True, download=True, transform=transforms)
test_set = datasets.FashionMNIST("dataset", train=False, download=True, transform=transforms)

from torch.utils.data import DataLoader
# train_loader = DataLoader(train_set, batch_size=512, shuffle=True)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False)

In [ ]:
from functools import reduce
from IPDL.optim.func import silverman_optimize
from IPDL.functional import matrix_estimator
def get_Ax(x, gamma=.5):
    n = x.size(0)
    d = x.size(1) if len(x.shape)==2 else reduce(lambda x,y:x*y, x.shape[1:])

    sigma = silverman_optimize(x, gamma=gamma, normalize=True)
    print(sigma)
    _, Ax = matrix_estimator(x, sigma)
    return Ax

In [ ]:
val_inputs = next(iter(test_loader))[0]
gamma = 4e-1
Ax = get_Ax(val_inputs, gamma=gamma)

In [ ]:
from IPDL.InformationTheory import MatrixBasedRenyisEntropy as MRE

ref_entropy = MRE.entropy(Ax)

print("Input Entropy: {}".format(ref_entropy))

In [ ]:
import pandas as pd
from IPDL.utils import show_aeip
from matplotlib import pyplot as plt
data = ["ae_dropout_p5_FMNIST", "ae_dropout_p2_FMNIST", "ae_no_dropout_FMNIST"]
for ae_data in data:
    df = pd.read_pickle(os.path.join(os.path.join("../Result/IP/", ae_data+".pkl")))
    ref_entropy = df.max().max()
    fig = show_aeip(df, ref_entropy, n=15, moving_average_n=8)
    fig.savefig(os.path.join("../Result/IP/imgs/", ae_data+".pdf"), bbox_inches='tight')
    plt.show()

df.head()

In [ ]:
df.max().mean()

# Diabetic Foot Ulcer

In [ ]:
from DFU import DiabeticFootDataset
# Diabetic Foot Dataset
    # Training dataset
DATASET_DIR = "/home/abian/Data/Dataset/DiabeticFootDataset/"
iactec_dataset_dir = os.path.join(DATASET_DIR, 'IACTEC/Processed/Cropped')
inaoe_dataset_dir = os.path.join(DATASET_DIR, 'INAOE/Processed/Cropped')
_dataset = DiabeticFootDataset([iactec_dataset_dir, inaoe_dataset_dir])

    # Testing dataset
test_dataset_dir = os.path.join(DATASET_DIR, 'Test/Processed/Cropped')
_test_set = DiabeticFootDataset(test_dataset_dir)

from DFU import DiabeticFootTorchDataset
from torchvision.transforms import ToTensor, Resize, Compose, Normalize
from torch.utils.data import DataLoader

# transforms = Compose([Resize((64, 64)), ToTensor()])
transform = Compose([Resize((64, 64)), ToTensor(), Normalize([0.5], [0.5])])

#train
train_set = DiabeticFootTorchDataset(_dataset, transform=transform)
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)

#test
test_set = DiabeticFootTorchDataset(_test_set, transform=transform)
val_loader = DataLoader(test_set, batch_size=32, shuffle=False)

In [ ]:
val_inputs = next(iter(val_loader))[0]
gamma = 4e-1
Ax = get_Ax(val_inputs, gamma=gamma)

In [ ]:
from IPDL.InformationTheory import MatrixBasedRenyisEntropy as MRE
ref_entropy = MRE.entropy(Ax)
print("Input Entropy: {}".format(MRE.entropy(Ax)))

In [ ]:
import pandas as pd
from IPDL.utils import show_aeip
from matplotlib import pyplot as plt
data = ["ae_dropout_p5_DFU", "ae_dropout_p2_DFU", "ae_no_dropout_DFU"]
ref_entropy = [1.5, 1.5, 2.8]
for idx, ae_data in enumerate(data):
    df = pd.read_pickle(os.path.join(os.path.join("../Result/IP/", ae_data+".pkl")))
    fig = show_aeip(df, ref_entropy[idx], n=18, moving_average_n=12)
    fig.savefig(os.path.join("../Result/IP/imgs/", ae_data+".pdf"), bbox_inches='tight')
    plt.show()


In [ ]:
import pandas as pd 
# df = pd.read_pickle(os.path.join("../Result/IP/ae_dropout_p5_DFU.pkl"))
# df = pd.read_pickle(os.path.join("../Result/IP/ae_dropout_p2_DFU.pkl"))
df = pd.read_pickle(os.path.join("../Result/IP/ae_no_dropout_DFU.pkl"))
df.head()

In [ ]:
from IPDL.utils import show_aeip
from matplotlib import pyplot as plt
fig = show_aeip(df, ref_entropy, n=18, moving_average_n=15)
plt.show()